# Step 4: Visualization

In [ ]:
%reset -f
import pycompwa.ui as pwa
pwa.Logging('error')  # at this stage we are not interested in the back-end
particle_list = pwa.read_particles('model.xml')
kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)
kinematics.create_all_subsystems()
data_sample = pwa.read_root_data(input_file='generated_data.root')
phsp_sample = pwa.read_root_data(input_file='generated_phsp.root')
intensity_builder = pwa.IntensityBuilderXML(
    'model.xml', particle_list, kinematics, phsp_sample)
fit_result = pwa.load('fit_result.xml')
intensity = intensity_builder.create_intensity()
data_set = kinematics.convert(data_sample)
estimator, initial_parameters = \
    pwa.create_unbinned_log_likelihood_function_tree_estimator(
        intensity, data_set)
intensity.updateParametersFrom(fit_result.final_parameters)

## 4.1 Create Dalitz plots

Let's go ahead and make a Dalitz plot of the data sample and our fit result. It is easiest to visualize your data with [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). This section illustrates how to do this in combination with ComPWA's functionality.

Since our model only includes one specific decay topology, the `HelicityKinematics` class only generates the kinematic variables needed to evaluate the `Intensity`. In order to make a Dalitz plot, the kinematic variables from the other subsystems are also needed. They can be created with `create_all_subsystems()`. The conversion from the event samples have to be performed again, though:

In [ ]:
kinematics.create_all_subsystems()
data_set = kinematics.convert(data_sample)
phsp_set = kinematics.convert(phsp_sample)

The `data_set` and `phsp_set` instances are `DataSet` objects can cannot be understood by pandas. Their data members, however, easily allow us to make the conversion:

In [ ]:
import pandas as pd
frame_data = pd.DataFrame(data_set.data)
frame_phsp = pd.DataFrame(phsp_set.data)
frame_data['weights'] = data_set.weights
frame_data

We added a weights column here as well, though in this case, all data weights are unity.

To change the number IDs here to the original particle names, you can extract a mapping from the `Kinematics` object:

In [ ]:
transition_info = kinematics.get_particle_state_transition_kinematics_info()
id_to_name = transition_info.get_final_state_id_to_name_mapping()
name_to_id = {val: key for key, val in id_to_name.items()}


def replace_ids(title):
    """Just some replace lambda"""
    for id, name in id_to_name.items():
        title = title.replace(str(id), name)
    return title

How to visualize the fit? This is where the phase space comes in again. By attributing an intensity as a weight to each phase space point, we get a point distribution of the fit model over the space spanned by the kinematic variables.

In [ ]:
intensity_set = intensity.evaluate(phsp_set.data)
frame_phsp['weights'] = intensity_set

Now you can apply the usual `matplotlib.pyplot` tools to plot the distributions.

In [ ]:
import matplotlib.pyplot as plt


def plot_1d_comparison(name, **kwargs):
    """Helper function for comparing the 1D distributions of fit and data"""
    frame_data[name].hist(bins=100, density=True,
                          alpha=.5, label='data', **kwargs)
    frame_phsp[name].hist(bins=100, weights=frame_phsp['weights'],
                          density=True, histtype='step', color='red',
                          label='fit', **kwargs)
    plt.ylabel('normalized intensity')
    title = replace_ids(name)
    plt.xlabel(title)

In [ ]:
plot_1d_comparison('theta_2_4_vs_3')
plt.legend()

In [ ]:
plot_1d_comparison('mSq_(3,4)')
plt.legend()

Similarly, we can easily create a Dalitz plot:

In [ ]:
import matplotlib.pyplot as plt


def dalitz_plot(frame, mass_x, mass_y, **kwargs):
    """Helper function to create a Dalitz plot with meaningful axis titles"""
    plt.hist2d(
        frame[mass_x],
        frame[mass_y],
        weights=frame['weights'],
        **kwargs
    )
    plt.xlabel(replace_ids(mass_x))
    plt.ylabel(replace_ids(mass_y))

In [ ]:
from itertools import combinations
variable_names = data_set.data.keys()
invariant_masses = [var for var in variable_names
                    if var.startswith('mSq_')
                    and not var.endswith('(2,3,4)')]
for comb in combinations(invariant_masses, 2):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    plt.sca(axs[0])
    dalitz_plot(frame_data, *comb, bins=100)
    axs[0].set_title('data')
    plt.sca(axs[1])
    dalitz_plot(frame_phsp, *comb, bins=100)
    axs[1].set_title('fit')
    plt.show()

## 4.2 Calculate Fit Fractions

Fit fractions can be calculated using the ``fit_fractions_with_propagated_errors()`` function. It requires amplitude or ``Intensity`` components that can be extracted from the ``IntensityBuilderXML`` instance. A nested list of the component names has to be specified as well. These are the names specified in the component XML attributes. If the inner lists contain more than one component, these components will be added coherently. In this way you can calculate your own customized fit fractions.

All registered component names can be retrieved via ``get_all_component_names()``:

In [ ]:
intensity_builder.get_all_component_names()

Now specify which components you want to get from the builder.

In [ ]:
components = intensity_builder.create_intensity_components([
    ['coherent_J/psi_-1_to_gamma_-1+pi0_0+pi0_0'],
    ['J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;'],
    ['J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;'],
    ['J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;'],
    [
        'J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;',
        'J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;',
        'J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;',
    ],
])

The last step is to call the actual fit fraction calculation function ``fit_fractions_with_propagated_errors()``. Here, the first required argument is a list of component pairs. Each pair resembles the nominator and denominator of a fit fraction.

In [ ]:
fit_fractions = pwa.fit_fractions_with_propagated_errors([
    (components[1], components[0]),
    (components[2], components[0]),
    (components[3], components[0]),
    (components[4], components[0]),
], phsp_set, fit_result)

In [ ]:
print(fit_fractions)

That's it. You can check some of the other examples to learn about more detailed features of ComPWA.

And give us feedback or [contribute](https://compwa.github.io/contribute.html)! ;)